In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ais-baltic-sea/dirways_all_2018_2019.csv
/kaggle/input/ais-baltic-sea/validation_set_winter.csv
/kaggle/input/ais-baltic-sea/training_set.csv
/kaggle/input/ais-baltic-sea/validation_set_summer.csv


In [2]:
import numpy as np
import pandas as pd
import os
import time
#import matplotlib.pyplot as plt
# %matplotlib inline 
# from mpl_toolkits.basemap import Basemap  # import Basemap matplotlib toolkit

DATA_DIR = "/kaggle/input/ais-baltic-sea/"

# dtype_spec = {
#     'iceclass': str,
# }
ais = pd.read_csv(DATA_DIR + 'training_set.csv', index_col=0) #parse_dates=['timestamp'], dtype=dtype_spec

/tmp/ipykernel_64/791605484.py:14: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  ais = pd.read_csv(DATA_DIR + 'training_set.csv', index_col=0) #parse_dates=['timestamp'], dtype=dtype_spec


In [3]:
#Print headings and description about the dataset
ais.head()
#ais.describe()
#print(ais['course'].describe())

,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,vessel_type,vessel_max_speed,draft,power,dwt,iceclass
0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA


# PRE-PROCESSING****

In [4]:
import pandas as pd
import datetime

def preprocess(data):
    # Remove unnecessary columns
    columns_to_keep = ['lat', 'lon', 'speed', 'course', 'timestamp', 'mmsi']
    data = data[columns_to_keep]
    
    # Drop rows with any null values in their columns
    data = data.dropna()

    # Convert 'timestamp' column to datetime objects
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Convert datetime objects to UNIX timestamp (seconds since the Unix epoch)
    data['timestamp_unix'] = data['timestamp'].apply(lambda x: int(x.timestamp()))

    # Sort by 'MMSI' and 'timestamp' columns
    data = data.sort_values(by=['mmsi', 'timestamp'])

    # Calculate timestep (time difference) between consecutive messages for each MMSI
    data['timestep_seconds'] = data.groupby('mmsi')['timestamp'].diff().dt.total_seconds()
    
    # Define conditions for incrementing the path counter
    # first condition is when the time interval path of a ship goes over an hour
    # second condition is when there is a change in MMSI
    conditions = (data['timestep_seconds'] > 3600) | (data['mmsi'].ne(data['mmsi'].shift()))

    # Create path counter based on conditions
    data['path'] = conditions.cumsum()
    # Now ais DataFrame will have a new column 'path' which increments
    # whenever the conditions (time difference > 1 hour or MMSI changes) are met.
    
    # Group by 'path' and count the number of messages in each path
    path_counts = data.groupby('path').size()

    # Create a new DataFrame with 'path' as index and 'pathCount' as the count of messages
    path_count_table = pd.DataFrame({'pathCount': path_counts})
    path_count_table.head()
    # The index of path_count_table will automatically be the path numbers (since 'path' was used as index)
    # path_count_table now holds the path number as index and the count of messages (pathCount) as the result

    # Filter paths with lengths less than 24 or greater than 144
    filtered_paths = path_counts[(path_counts < 24) | (path_counts > 144)].index

    # Convert the index (path numbers) to an array
    filtered_paths_array = filtered_paths.to_numpy()
    # filtered_paths_array now holds the path numbers with lengths outside the specified range
    #print(filtered_paths_array)

    # Eliminate all invalid paths
    #'filtered_paths_array' contains the invalid path numbers as calculated previously
    # Step 1: Sort the filtered paths array
    filtered_paths_array = sorted(filtered_paths_array)
    # Step 2: Filter the ais_sorted DataFrame to remove rows with invalid paths
    data = data[~data['path'].isin(filtered_paths_array)]
    # Now ais_valid contains only valid paths
    #print(len(ais_valid))
    #print(len(ais))

    return data


# Preprocess the dataset
ais_valid = preprocess(ais)
ais_valid.head()
#print(ais_valid[ais_valid['mmsi'] == 205366000])
#print(ais_valid['path'].max())

,lat,lon,speed,course,timestamp,mmsi,timestamp_unix,timestep_seconds,path
922,57.756818,10.420237,9.16,78.0,2017-12-22 10:23:57,205366000,1513938237,1973822.0,4
923,57.767407,10.509043,8.90,77.1,2017-12-22 10:34:03,205366000,1513938843,606.0,4
924,57.777053,10.594123,8.18,83.0,2017-12-22 10:45:13,205366000,1513939513,670.0,4
925,57.776770,10.678505,8.33,92.8,2017-12-22 10:55:19,205366000,1513940119,606.0,4
926,57.768455,10.734610,3.55,156.2,2017-12-22 11:04:30,205366000,1513940670,551.0,4


# TRAIN-TEST-SPLIT****

In [5]:
#Now we use ais_valid to provide train,test and validation sets

import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Split the data into 80% train and 20% remaining
train_data, remaining_data = train_test_split(ais_valid, test_size=0.20, random_state=42)

# Step 2: Split the remaining 20% into 10% test and 10% validation
test_data, val_data = train_test_split(remaining_data, test_size=0.50, random_state=42)

# Now we have 80% train, 10% test, and 10% validation sets
print(f"Train data size: {train_data.shape}")
print(f"Test data size: {test_data.shape}")
print(f"Validation data size: {val_data.shape}")


Train data size: (891604, 9)
Test data size: (111450, 9)
Validation data size: (111451, 9)


# NORMALIZATION****

In [6]:
# Normalize values, specifically:
#   Lat = (lat - lat_min) / (lat_max-lat_min)
#   Lon = (lon - lon_min) / (lon_max-lon_min)
#   Speed = speed / 52.8
#   Course = course / 360

# Calculate min and max for latitude and longitude
lat_min = ais_valid['lat'].min()
lat_max = ais_valid['lat'].max()
lon_min = ais_valid['lon'].min()
lon_max = ais_valid['lon'].max()

# print(f"Latitude: min = {lat_min}, max = {lat_max}")
# print(f"Longitude: min = {lon_min}, max = {lon_max}")

# Define the normalization function
def normalize_data(df, lat_min, lat_max, lon_min, lon_max):
    df['alat'] = (df['lat'] - lat_min) / (lat_max - lat_min)
    df['alon'] = (df['lon'] - lon_min) / (lon_max - lon_min)
    df['aspeed'] = df['speed'] / 52.8
    df['acourse'] = df['course'] / 360
    return df

# Apply the normalization to train, test, and validation datasets
train_data = normalize_data(train_data, lat_min, lat_max, lon_min, lon_max)
test_data = normalize_data(test_data, lat_min, lat_max, lon_min, lon_max)
val_data = normalize_data(val_data, lat_min, lat_max, lon_min, lon_max)

#print(train_data.head())


# PATH INI: PROPER INDEXING FOR DICTIONARIES****

In [7]:
#For each set, create a new pathIndex called pathIni and start this index at 0,incrementing every time the path changes 
#this helps indexing in the dictionary later

# Function to create the pathIni index
def create_path_ini(df):
    df = df.sort_values(by='path').reset_index(drop=True)  # Ensure the data is sorted by 'path'
    unique_paths = df['path'].unique()  # Get unique paths
    path_mapping = {path: idx for idx, path in enumerate(unique_paths)}  # Create a mapping from old path to new index
    df['pathIni'] = df['path'].map(path_mapping)  # Map old paths to new pathIni indices
    return df

# Apply the function to each dataset
train_data = create_path_ini(train_data)
test_data = create_path_ini(test_data)
val_data = create_path_ini(val_data)

#print(train_data.head())
#print(test_data.head())
#print(val_data.head())

#train_data["pathIni"].max()
#test_data["pathIni"].max()
#val_data["pathIni"].max()

# TRANSFORMING FRAMES INTO CSV FILES****

In [8]:
train_data.to_csv("marinedata_train.csv", index=False)
test_data.to_csv("marinedata_test.csv", index=False)
val_data.to_csv("marinedata_val.csv", index=False)

# LOADING CSV FILES****

In [9]:
########################### Use proper datapath here
dataset_path = "/kaggle/working/"
l_csv_filename =["marinedata_train.csv",
                 "marinedata_test.csv",
                "marinedata_val.csv"]

#pkl_filename = "marinedata_train.pkl"
pkl_filename_train = "marinedata_train.pkl"
pkl_filename_test  = "marinedata_test.pkl"
pkl_filename_valid = "marinedata_val.pkl"

#Column indices for easier reference
ALAT, ALON, ASPEED, ACOURSE, TIMESTAMP, MMSI, PATH = list(range(7))

## LOADING CSV FILES
#======================================
m_msg_train = []
m_msg_test = []
m_msg_valid = []

for csv_filename in l_csv_filename:
    data_path = os.path.join(dataset_path,csv_filename)
    f = pd.read_csv(data_path)
    print("Reading ", csv_filename, "...")
    
    l_l_msg = [] # list of AIS messages, each row is a message (list of AIS attributes)
    for i in range(len(f["mmsi"])):
        row = f.iloc[i]
        l_l_msg.append([float(row["alat"]), float(row["alon"]), float(row["aspeed"]),
                            float(row["acourse"]),
                            int(row["timestamp_unix"]), int(row["mmsi"]),
                           int(row["pathIni"])])
    if csv_filename == "marinedata_train.csv":
        m_msg_train = np.array(l_l_msg)
        print("Total number of AIS messages: ",m_msg_train.shape[0])
    if csv_filename == "marinedata_test.csv":
        m_msg_test = np.array(l_l_msg)
        print("Total number of AIS messages: ",m_msg_test.shape[0])
    if csv_filename == "marinedata_val.csv":
        m_msg_valid = np.array(l_l_msg)
        print("Total number of AIS messages: ",m_msg_valid.shape[0])


## FILTERING 
#======================================
# Selecting AIS messages in the ROI and in the period of interest.

print("Number of msgs in the training set: ",len(m_msg_train))
print("Number of msgs in the test set: ",len(m_msg_test))
print("Number of msgs in the validation set: ",len(m_msg_valid))

## MERGING INTO DICT
print("Convert to dicts of vessel's tracks...")

Reading  marinedata_train.csv ...
Total number of AIS messages:  891604
Reading  marinedata_test.csv ...
Total number of AIS messages:  111450
Reading  marinedata_val.csv ...
Total number of AIS messages:  111451
Number of msgs in the training set:  891604
Number of msgs in the test set:  111450
Number of msgs in the validation set:  111451
Convert to dicts of vessel's tracks...


# CONVERT MESSAGES TO DICTIONARIES****

In [10]:
from tqdm import tqdm                #tqdm can be used to display progress bars

# Training set
Vs_train = dict()
for v_msg in tqdm(m_msg_train):
    mmsi = int(v_msg[MMSI])
    pathIni = int(v_msg[PATH])
    if not (pathIni in list(Vs_train.keys())):
        Vs_train[pathIni] = {'mmsi': mmsi, 'traj': np.empty((0,6))}
    Vs_train[pathIni] = {'mmsi': mmsi, 'traj': np.concatenate((Vs_train[pathIni]['traj'], np.expand_dims(v_msg[:6],0)), axis = 0)}
for key in tqdm(list(Vs_train.keys())):
    mmsi = int(Vs_train[key]['traj'][0][5])
    Vs_train[key] = {'mmsi': mmsi,'traj': np.array(sorted(Vs_train[key]['traj'], key=lambda m_entry: m_entry[TIMESTAMP]))}

# Validation set
Vs_valid = dict()
for v_msg in tqdm(m_msg_valid):
    mmsi = int(v_msg[MMSI])
    pathIni = int(v_msg[PATH])
    if not (pathIni in list(Vs_valid.keys())):
        Vs_valid[pathIni] = {'mmsi': mmsi, 'traj': np.empty((0,6))}
    Vs_valid[pathIni] = {'mmsi': mmsi, 'traj': np.concatenate((Vs_valid[pathIni]['traj'], np.expand_dims(v_msg[:6],0)), axis = 0)}
for key in tqdm(list(Vs_valid.keys())):
    mmsi = int(Vs_valid[key]['traj'][0][5])
    Vs_valid[key] = {'mmsi': mmsi,'traj': np.array(sorted(Vs_valid[key]['traj'], key=lambda m_entry: m_entry[TIMESTAMP]))}

# Test set
Vs_test = dict()
for v_msg in tqdm(m_msg_test):
    mmsi = int(v_msg[MMSI])
    pathIni = int(v_msg[PATH])
    if not (pathIni in list(Vs_test.keys())):
        Vs_test[pathIni] = {'mmsi': mmsi, 'traj': np.empty((0,6))}
    Vs_test[pathIni] = {'mmsi': mmsi, 'traj': np.concatenate((Vs_test[pathIni]['traj'], np.expand_dims(v_msg[:6],0)), axis = 0)}
for key in tqdm(list(Vs_test.keys())):
    mmsi = int(Vs_test[key]['traj'][0][5])
    Vs_test[key] = {'mmsi': mmsi,'traj': np.array(sorted(Vs_test[key]['traj'], key=lambda m_entry: m_entry[TIMESTAMP]))}



100%|██████████| 14612/14612 [00:00<00:00, 64486.07it/s]


# PICKLING****

In [11]:
# SAVING TO PICKLE FILES
#======================================
import pickle

with open(pkl_filename_train, 'wb') as f:
    pickle.dump(m_msg_train, f)
    print(f"Training data saved to {pkl_filename_train}")

with open(pkl_filename_test, 'wb') as f:
    pickle.dump(m_msg_test, f)
    print(f"Test data saved to {pkl_filename_test}")

with open(pkl_filename_valid, 'wb') as f:
    pickle.dump(m_msg_valid, f)
    print(f"Validation data saved to {pkl_filename_valid}")

Training data saved to marinedata_train.pkl
Test data saved to marinedata_test.pkl
Validation data saved to marinedata_val.pkl
